In [1]:
!pip install --upgrade pip setuptools wheel
!pip install gym==0.26.2
!pip install ale-py==0.7.4
!pip install autorom[accept-rom-license] --no-cache-dir
!pip install "ray[rllib]" matplotlib

  Using cached setuptools-80.4.0-py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 14.3 MB/s eta 0:00:00
Using cached setuptools-80.4.0-py3-none-any.whl (1.2 MB)



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: To modify pip, please run the following command:
C:\Users\Sara Iqbal\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip setuptools wheel



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement ale-py==0.7.4 (from versions: 0.9.0, 0.9.1, 0.10.0, 0.10.1, 0.10.2, 0.11.0)

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for ale-py==0.7.4


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for AutoROM.accept-rom-license: filename=autorom_accept_rom_license-0.6.1-py3-none-any.whl size=446730 sha256=44b0ba0bce80c285b3bca06967f3b632e4ffcf0d8a6b66ca0389ded8b4de4ea2
  Stored in directory: C:\Users\Sara Iqbal\AppData\Local\Temp\pip-ephem-wheel-cache-5um4e5tc\wheels\99\f1\ff\c6966c034a8259164bdc9deb4d1ea839f119474638100e6645
Successfully built AutoROM.accept-rom-license



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Import Required Libraries

import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
import matplotlib.pyplot as plt
import os
import json
import numpy as np

In [ ]:
# Initialize Ray and Configure PPO for Pong

ray.shutdown() # restart Ray if already running
ray.init(ignore_reinit_error=True)

config = (
PPOConfig()
.environment("PongNoFrameskip-v4")
.framework("torch")
.rollouts(num_rollout_workers=2)
.training(
gamma=0.99,
lr=5e-4,
train_batch_size=4000,
)
.resources(num_gpus=0) # set to 1 if running on GPU
)

2025-05-11 15:39:24,149	ERROR services.py:1362 -- Failed to start the dashboard , return code 3221226505
2025-05-11 15:39:24,154	ERROR services.py:1387 -- Error should be written to 'dashboard.log' or 'dashboard.err'. We are printing the last 20 lines for you. See 'https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#logging-directory-structure' to find where the log file is.
2025-05-11 15:39:24,157	ERROR services.py:1431 -- 
The last 20 lines of C:\Users\SARAIQ~1\AppData\Local\Temp\ray\session_2025-05-11_15-39-13_301673_34884\logs\dashboard.log (it contains the error message from the dashboard): 
Traceback (most recent call last):
  File "C:\Users\Sara Iqbal\AppData\Local\Programs\Python\Python312\Lib\site-packages\ray\dashboard\dashboard.py", line 247, in <module>
    logging_utils.redirect_stdout_stderr_if_needed(
  File "C:\Users\Sara Iqbal\AppData\Local\Programs\Python\Python312\Lib\site-packages\ray\_private\logging_utils.py", line 48, in redirect_st

In [ ]:
# Train the PPO Agent (20 Iterations)

tuner = tune.Tuner(
"PPO",
param_space=config.to_dict(),
run_config=air.RunConfig(
stop={"training_iteration": 20},
checkpoint_config=air.CheckpointConfig(checkpoint_at_end=True),
verbose=1,
name="PPO_Pong_Training",
),
)
results = tuner.fit()

In [ ]:
# Extract Training Results

result_dir = results.get_best_result().log_dir
metrics_path = os.path.join(result_dir, "result.json")

iterations = []
mean_rewards = []
reward_variance = []

with open(metrics_path, "r") as f:
for line in f:
data = json.loads(line)
if "episode_reward_mean" in data:
iterations.append(data["training_iteration"])
mean_rewards.append(data["episode_reward_mean"])
reward_variance.append(np.var(data.get("hist_stats", {}).get("episode_reward", [0])))

In [ ]:
#STEP 6: Plot Mean Episode Reward

plt.figure(figsize=(10, 5))
plt.plot(iterations, mean_rewards, marker='o', label="Mean Reward")
plt.xlabel("Training Iteration")
plt.ylabel("Mean Episode Reward")
plt.title("PPO on Pong – Mean Episode Reward")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
#STEP 7: Plot Reward Variance

plt.figure(figsize=(10, 5))
plt.plot(iterations, reward_variance, marker='x', color='orange', label="Reward Variance")
plt.xlabel("Training Iteration")
plt.ylabel("Reward Variance")
plt.title("PPO on Pong – Reward Variance")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
#STEP 8: Clean Up 
ray.shutdown()
